In [ ]:
from scipy.stats import spearmanr, pearsonr

def calculate_ic_metrics(y_true, y_pred):
    ic, _ = pearsonr(y_true, y_pred)
    rank_ic, _ = spearmanr(y_true, y_pred)
    print(f"Information Coefficient (IC): {ic:.4f}")
    print(f"Rank IC: {rank_ic:.4f}")
    return ic, rank_ic

# 사용 예시
calculate_ic_metrics(y_test, y_pred)

In [4]:
import numpy as np
import pandas as pd

def get_rank_ic(factor, target):
    """
    Rank IC를 계산하는 함수 (scipy 없이 구현)
    
    Args:
        factor (pd.Series or np.array): 예측 팩터 값
        target (pd.Series or np.array): 실제 수익률 값 (또는 라벨)
        
    Returns:
        float: Rank IC 값 (-1 ~ 1)
    """
    # 데이터가 리스트나 numpy array일 경우를 대비해 Series로 변환
    if not isinstance(factor, pd.Series):
        factor = pd.Series(factor)
    if not isinstance(target, pd.Series):
        target = pd.Series(target)
        
    # 1. 결측치(NaN) 제거 (둘 중 하나라도 NaN이면 제외)
    # 실제 퀀트 데이터에는 NaN이 많으므로 안전장치로 넣는 것이 좋습니다.
    valid_idx = (~factor.isna()) & (~target.isna())
    f_clean = factor[valid_idx]
    t_clean = target[valid_idx]
    
    # 2. 값들을 순위(Rank)로 변환
    # method='average'는 동점자 처리 시 평균 순위를 부여 (가장 일반적)
    f_rank = f_clean.rank(method='average')
    t_rank = t_clean.rank(method='average')
    
    # 3. 순위 간의 피어슨 상관계수 계산 (np.corrcoef)
    # np.corrcoef는 공분산 행렬을 반환하므로 [0, 1] 요소를 가져옴
    ic = np.corrcoef(f_rank, t_rank)[0, 1]
    
    return ic

pred_factor = [0.1, 0.5, 0.2, 0.9, 0.4]  # 내가 만든 팩터
real_return = [0.05, 0.12, 0.04, 0.20, 0.08] # 실제 수익률

ic_score = get_rank_ic(pred_factor, real_return)

print(f"Calculated Rank IC: {ic_score:.4f}")

Calculated Rank IC: 0.9000


In [ ]:
def plot_decile_analysis(y_true, y_pred, title="Decile Analysis"):
    df_eval = pd.DataFrame({'Actual': y_true, 'Pred': y_pred})
    
    # 예측값 기준으로 10개 그룹으로 나누기 (qcut)
    df_eval['Decile'] = pd.qcut(df_eval['Pred'], 10, labels=False)
    
    # 각 그룹별 실제 값의 평균 계산
    decile_means = df_eval.groupby('Decile')['Actual'].mean()
    
    # 시각화
    plt.figure(figsize=(10, 6))
    decile_means.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.title(f'{title} (Monotonicity Check)')
    plt.xlabel('Predicted Decile (0=Lowest, 9=Highest)')
    plt.ylabel('Average Actual Value')
    plt.grid(axis='y', linestyle='--')
    plt.show()

# 사용 예시
plot_decile_analysis(y_test, y_pred)

In [ ]:
def plot_residuals(y_true, y_pred):
    residuals = y_true - y_pred
    
    fig, ax = plt.subplots(1, 2, figsize=(15, 6))
    
    # 1. Residuals vs Predicted (이분산성 체크)
    sns.scatterplot(x=y_pred, y=residuals, alpha=0.3, ax=ax[0])
    ax[0].axhline(0, color='red', linestyle='--')
    ax[0].set_title('Residuals vs Predicted')
    ax[0].set_xlabel('Predicted Value')
    ax[0].set_ylabel('Residuals (Error)')
    
    # 2. Residual Distribution (정규성 체크)
    sns.histplot(residuals, kde=True, ax=ax[1])
    ax[1].set_title('Distribution of Residuals')
    
    plt.show()

# 사용 예시
plot_residuals(y_test, y_pred)

In [ ]:
def plot_rolling_performance(y_true, y_pred, dates, window=3):
    # 날짜 인덱스가 있는 데이터프레임 생성
    df_eval = pd.DataFrame({'Actual': y_true, 'Pred': y_pred}, index=dates)
    
    # 월별(혹은 특정 기간별) RMSE 계산
    # 'M'은 월말 기준. 데이터 빈도에 따라 'D', 'H' 등으로 변경
    monthly_rmse = df_eval.resample('M').apply(
        lambda x: np.sqrt(mean_squared_error(x['Actual'], x['Pred']))
    )
    
    plt.figure(figsize=(12, 6))
    plt.plot(monthly_rmse.index, monthly_rmse, marker='o', linestyle='-')
    plt.title('Rolling RMSE over Time (Stability Check)')
    plt.xlabel('Date')
    plt.ylabel('RMSE')
    plt.show()

# 사용 예시 (X_test에 날짜 인덱스가 있거나 별도로 저장해뒀어야 함)
# plot_rolling_performance(y_test, y_pred, X_test_dates)